# Multi-task optimization of KB mirrors

Often, we want to optimize multiple aspects of a system; in this real-world example aligning the Kirkpatrick-Baez mirrors at the TES beamline's endstation, we care about the horizontal and vertical beam size, as well as the flux. 

We could try to model these as a single task by combining them into a single number (i.e., optimization the beam density as flux divided by area), but our model then loses all information about how different inputs affect different outputs. We instead give the optimizer multiple "tasks", and then direct it based on its prediction of those tasks. 

In [ ]:
%run -i ../../../examples/prepare_bluesky.py
%run -i ../../../examples/prepare_tes_shadow.py

kb_dofs = [kbv.x_rot, kbh.x_rot]
kb_bounds = np.array([[-0.10, +0.10], [-0.10, +0.10]]) 

In [ ]:

from bloptools.bayesian import Agent
from bloptools.experiments.sirepo.tes import digestion
from bloptools.tasks import Task

beam_flux_task = Task(key="flux", kind="max", transform=lambda x: np.log(x))
beam_width_task = Task(key="x_width", kind="min", transform=lambda x: np.log(x))
beam_height_task = Task(key="y_width", kind="min", transform=lambda x: np.log(x))

boa = Agent(
       active_dofs=kb_dofs, 
       active_dof_bounds=kb_bounds,
       detectors=[w9],
       tasks=[beam_flux_task, beam_width_task, beam_height_task],
       digestion=digestion,
       db=db,
)

RE(boa.initialize(init_scheme='quasi-random', n_init=2))

For each task, we plot the sampled data and the model's posterior with respect to two inputs to the KB mirrors. We can see that each tasks responds very differently to different motors, which is very useful to the optimizer. 

In [ ]:
boa.plot_tasks()

Let's optimize:

In [ ]:
RE(boa.learn(strategy='esti', n_iter=4))
boa.plot_tasks()

The agent has learned that certain dimensions affect different tasks differently!